In [0]:
CATALOG = dbutils.widgets.get("CATALOG")
REFUNDMANAGER_LAKEBASE_INSTANCE_NAME = f"{CATALOG}refundmanager"

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance
from databricks.sdk.errors import NotFound, ResourceDoesNotExist
import uuid
import time

import sys
sys.path.append('../utils')
from uc_state import add

w = WorkspaceClient()

instance_name = REFUNDMANAGER_LAKEBASE_INSTANCE_NAME

# Idempotent: reuse existing instance or create new one
try:
    existing = w.database.get_database_instance(instance_name)
    print(f"♻️ Found existing database instance: {existing.name} (state: {existing.state})")
    instance = existing
except (NotFound, ResourceDoesNotExist):
    print(f"Creating new database instance: {instance_name}")
    instance = w.database.create_database_instance(
        DatabaseInstance(
            name=instance_name,
            capacity="CU_1"
        )
    )
    print(f"Created database instance: {instance.name}")
    add(CATALOG, "databaseinstances", {"name": instance.name})

# Block until it's actually ready
print("Waiting for database instance to be available...")
for _ in range(60):
    state = str(w.database.get_database_instance(instance_name).state)
    if state == "DatabaseInstanceState.AVAILABLE":
        print(f"Instance ready: {state}")
        break
    time.sleep(10)
else:
    raise TimeoutError(f"Database instance {instance_name} did not become available within 10 minutes")



In [ ]:
from databricks.sdk.service.database import DatabaseCatalog

catalog_name = REFUNDMANAGER_LAKEBASE_INSTANCE_NAME

try:
    existing_cat = w.catalogs.get(catalog_name)
    print(f"♻️ Found existing catalog: {existing_cat.name}")
except Exception:
    catalog = w.database.create_database_catalog(
        DatabaseCatalog(
            name=catalog_name,
            database_instance_name=REFUNDMANAGER_LAKEBASE_INSTANCE_NAME,
            database_name="caspers",
            create_database_if_not_exists=True
        )
    )
    print(f"Created new database and catalog: {catalog.name}")
    add(CATALOG, "databasecatalogs", catalog)

In [0]:
from databricks.sdk.service.database import SyncedDatabaseTable, SyncedTableSpec, NewPipelineSpec, SyncedTableSchedulingPolicy

synced_table_name = f"{CATALOG}.recommender.pg_recommendations"

try:
    existing_table = w.tables.get(synced_table_name)
    print(f"♻️ Found existing synced table: {existing_table.full_name}")
except Exception:
    synced_table = w.database.create_synced_database_table(
        SyncedDatabaseTable(
            name=synced_table_name,
            database_instance_name=instance_name,
            logical_database_name="caspers",
            spec=SyncedTableSpec(
                source_table_full_name=f"{CATALOG}.recommender.refund_recommendations",
                primary_key_columns=["order_id"],
                scheduling_policy=SyncedTableSchedulingPolicy.CONTINUOUS,
                create_database_objects_if_missing=True,
                new_pipeline_spec=NewPipelineSpec(
                    storage_catalog="storage_catalog",
                    storage_schema="storage_schema"
                )
            ),
        )
    )
    print(f"Created synced table: {synced_table.name}")
    add(CATALOG, "pipelines", synced_table.data_synchronization_status)